In [1]:
#!pip install gldb[tutorial]


## Data Stores

The database shall interact with "**data stores**" (here used as a more generic word for database). Through them, data can be accessed (uploaded and queried).

They can be databases for raw or metadata.

Let's implement the concrete implementations for a CSV and a in-memory-RDF database.

In [ ]:
from gldb.query import Query
import pathlib

import pandas as pd
import rdflib

In [5]:
import gldb

## Data Stores

The
database
shall
interact
with "**data stores**"(
        here used as a more generic word for database).Through them, data can be accessed (uploaded and queried).

They
can
be
databases
for raw or metadata.

Let
's implement the concrete implementations for a CSV and a in-memory-RDF database.

In [ ]:
from gldb.query import Query, QueryResult
from gldb.stores import DataStore, RDFStore
import pathlib

import pandas as pd
import rdflib


In [7]:
import gldb


## Data Stores

The
database
shall
interact
with "**data stores**"(
        here used as a more generic word for database).Through them, data can be accessed (uploaded and queried).

They
can
be
databases
for raw or metadata.

Let
's implement the concrete implementations for a CSV and a in-memory-RDF database.

In [9]:
from gldb.query import Query


## Data Stores

The
database
shall
interact
with "**data stores**"(
        here used as a more generic word for database).Through them, data can be accessed (uploaded and queried).

They
can
be
databases
for raw or metadata.

Let
's implement the concrete implementations for a CSV and a in-memory-RDF database.

In [ ]:
from gldb.query import Query, QueryResult
from gldb.stores import DataStore, RDFStore

In [11]:
class CSVQuery(Query):

    def __init__(self, query, table):
        self._query = query
        self._table = table

    def execute(self, store: DataStore, *args, **kwargs):
        return QueryResult(query=self, data=store.tables[self._table].query(self._query))

In [ ]:
class CSVDataStore(DataStore):

    def __init__(self, filenames=None):
        self._filenames = filenames or []
        self.tables = {}

    def upload_file(self, filename: pathlib.Path) -> bool:
        if filename.resolve().absolute() in self._filenames:
            return True
        self._filenames.append(filename.resolve().absolute())
        self.tables[filename.stem] = pd.read_csv(filename)
        return True

In [12]:
class InMemoryRDFDataStore(RDFStore):

    def __init__(self):
        self._filenames = []
        self._graphs = {}

    def upload_file(self, filename: pathlib.Path) -> bool:
        self._filenames.append(filename.resolve().absolute())
        return True

    @property
    def graph(self) -> rdflib.Graph:
        """Return graph for the metadata store."""
        combined_graph = rdflib.Graph()
        for filename in self._filenames:
            g = self._graphs.get(filename, None)
            if not g:
                g = rdflib.Graph()
                g.parse(filename)
                for s, p, o in g:
                    if isinstance(s, rdflib.BNode):
                        new_s = rdflib.URIRef(f"https://example.org/{s}")
                    else:
                        new_s = s
                    if isinstance(o, rdflib.BNode):
                        new_o = rdflib.URIRef(f"https://example.org/{o}")
                    else:
                        new_o = o
                    g.remove((s, p, o))
                    g.add((new_s, p, new_o))
                self._graphs[filename] = g
            combined_graph += g
        return combined_graph

## The Database instance

The core implementation concerns the implementation of `GenericLinkedDatabase`:

In [ ]:
db = gldb.GenericLinkedDatabase(
    {
        "csv": CSVDataStore(),
        "rdf": InMemoryRDFDataStore()
    }
)

Popoulate the stores with data:

In [14]:
for filename in pathlib.Path("data").glob('*.jsonld'):
    db.stores.rdf.upload_file(filename)

In [15]:
for filename in pathlib.Path("data").glob('*.csv'):
    db.stores.csv.upload_file(filename)

## Query the RDF store

Every store as a property `query` which returns the query-object of the store. In case of the already implemented `RDFStore` it is the `SparqlQuery` (also implemented by `gldb`):

Let's formulate the SPARQL query string:

In [17]:
from gldb.query import SparqlQuery

,file,person
0,file:temperature.csv,https://example.org/N54a176148a7a4671bee2166e4...


In [ ]:
query = SparqlQuery("""
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
SELECT * WHERE {
    ?file a dcat:Dataset .
    ?file dcterm:creator ?person .
    ?person a foaf:Person .
}
""")

Perform the query:

In [19]:
res = query.execute(db.stores.rdf)

,timestamp,temperature
4,2024-01-01 04:00,23.2
5,2024-01-01 05:00,23.8
6,2024-01-01 06:00,24.9
7,2024-01-01 07:00,25.5
8,2024-01-01 08:00,24.0


In [ ]:
res.data

## Query the CSV store

In [21]:
csv_query = CSVQuery("temperature > 23.0", "temperature")

In [22]:
csv_res = csv_query.execute(db.stores.csv)
csv_res.data

,timestamp,temperature
4,2024-01-01 04:00,23.2
5,2024-01-01 05:00,23.8
6,2024-01-01 06:00,24.9
7,2024-01-01 07:00,25.5
8,2024-01-01 08:00,24.0


# Federated queries

Since raw temperature data is stored in a different database (store) than the metadata, there is the need to combine the data.

This must currently be done using custom functions that return a `FederatedQueryResult`:

In [ ]:
def fetch_temperature_dataset(query, table) -> gldb.query.FederatedQueryResult:
    """custom federated query"""
    csv_query = CSVQuery(query, table)
    csv_res = csv_query.execute(db.stores.csv)

    query_all_metadata_of_temperature = f"""
    PREFIX dcat: <http://www.w3.org/ns/dcat#>
    SELECT * WHERE {{
        <file:{table}.csv> a dcat:Dataset .
        ?s ?p ?o
    }}
    """
    query = SparqlQuery(query_all_metadata_of_temperature)
    rdf_res = query.execute(db.stores.rdf)

    fed_res = gldb.query.FederatedQueryResult(
        data=csv_res.data,
        metadata=rdf_res.data
    )
    return fed_res

In [ ]:
federated_result = fetch_temperature_dataset(query="temperature > 23.0", table="temperature")

In [ ]:
federated_result.data

In [ ]:
federated_result.metadata